<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/WIGAN_Implementation_from_scratch_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#seting-up the CoLaB environment and the GPU device if available
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>>You are on CoLaB with torch version: {torch.__version__}")
except Exception as e:
  print(f">>>>{type(e)}: {e}\n>>>>please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
def time_fmt(t: float = 126.891)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h} hrs: {m:>02} min: {s:>05.2f} sec"
print(f">>>>time formating:\tplease wait.....\n>>>>time elapsed is\t{time_fmt()}")

Mounted at /content/drive
>>>>You are on CoLaB with torch version: 1.8.1+cu101
>>>>time formating:	please wait.....
>>>>time elapsed is	0 hrs: 02 min: 06.00 sec


In [2]:
#In this notebook we are going to train an improved DCGAN using Wasserstein Loss
#On the nutshell this is an implementantion of WGAN.

In [3]:
#Importing modules and set-up the seed values for reproducability 
# we also setting the gpu to deterministic to avoid errors during training


In [4]:
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import time, random, math


In [5]:
seed = 1234
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [6]:
#This model is a typical GAN architecture with the few difference on the loss function and training loop
#We have both the Generator and the discriminator as in a ussual DCGAN (Using CNN architecture)
#For demonstration we are going to use MNIST images but any images can be imployed here:

In [7]:
#The following class build the discriminator network:
class Discriminator(nn.Module):
  def __init__(self, img_channels, d_features):
    super(Discriminator, self).__init__()
    self.discriminator = nn.Sequential(
        nn.Conv2d(img_channels,
                  d_features,
                  kernel_size = 4,
                  stride = 2,
                  padding = 1),
        nn.LeakyReLU(0.2),
        self.__dblock__(d_features, 2*d_features, 4, 2, 1),
        self.__dblock__(2*d_features, 4*d_features, 4,2,1),
        self.__dblock__(4*d_features, 8*d_features, 4,2,1),
        nn.Conv2d(8*d_features, 1, kernel_size = 4, stride = 2, padding = 0))
  def __dblock__(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias = False
        ),
      nn.BatchNorm2d(out_channels),
      nn.LeakyReLU(0.2))
  
  def forward(self, input_tensor):
    return self.discriminator(input_tensor)

In [8]:
#The following class build the generator network:
class Generator(nn.Module):
  def __init__(self, z_dim, in_channels, g_features):
    super(Generator, self).__init__()
    self.generator = nn.Sequential(
        self.__gblock__(z_dim, g_features*16, 4, 2, 0),
        self.__gblock__(g_features*16, g_features*8, 4,2,1),
        self.__gblock__(g_features*8, g_features*4, 4,2,1),
        self.__gblock__(g_features*4, g_features*2, 4, 2,1),
        nn.ConvTranspose2d(2*g_features, in_channels, kernel_size = 4, stride = 2, padding = 1),
        nn.Tanh())

  def __gblock__(self, in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.ConvTranspose2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            bias = False
        ),
      nn.BatchNorm2d(out_channels),
      nn.ReLU()
    )
  
  def forward(self, input_tensor):
    return self.generator(input_tensor)
  

In [9]:
#We can test the above classes if they yield the intended outputs:
#The generator must give an image as output and the discriminator must
#give a probability (sigmoid) to classify the input as fake/real

In [10]:
def __test__():
  img_channels = 3
  batch_size = 64
  z_dim = 100
  H, W = 64, 64
  rand_noise = torch.randn((batch_size, z_dim, 1,1))
  rand_img = torch.randn((batch_size,img_channels,H,W))
  disc = Discriminator(img_channels, 8)
  gen = Generator(z_dim, img_channels, 8)
  gen_out = gen(rand_noise)
  disc_out = disc(rand_img)
  return gen_out.shape, disc_out.shape

In [11]:
__test__()

(torch.Size([64, 3, 64, 64]), torch.Size([64, 1, 1, 1]))

In [12]:
#We initialize the parameters to random normal distribution with mean 0 and std 0.01
def __par__(model):
  for m in model.modules():
    if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
      nn.init.normal_(m.weight.data, mean = 0.00, std = 0.01)

In [13]:
#Hyperparameters to used for training this model:
learning_rate = 5e-5
batch_size = 64
img_size = 64
z_dim = 100
g_features = 64
d_features = 64
img_channels = 1#may be any number of channel..works with rgb too
EPOCHS = 10
clip_interval = 0.01
disc_iter = 5


In [14]:
#Instantiate the model class and initialize the weights using the above function:
generator = Generator(z_dim, img_channels,g_features).to(device = device)
discriminator = Discriminator(img_channels, d_features).to(device = device)
__par__(generator)
__par__(discriminator)

In [15]:
#Loading and preprocessing the data using transforms module:
transforms = transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Resize(img_size),
                                 transforms.Normalize(
                                     [0.5 for _ in range(img_channels)], [0.5 for _ in range(img_channels)]
                                 )])
dfm = datasets.MNIST(root = "mnist_data/",transform = transforms, download = True)
loader = DataLoader(dataset = dfm, shuffle = True, batch_size = batch_size)
x_dfm, y_dfm = next(iter(loader))
print(f"x_dfm_shape: {x_dfm.shape}\ty_dfm_shape: {y_dfm.shape}")

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw




Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


x_dfm_shape: torch.Size([64, 1, 64, 64])	y_dfm_shape: torch.Size([64])


In [16]:
#Get the optimizers objects:
#Note: we are going to use RMSprop optimizer exclusively for better results as suggested in WGAN paper
#We are going to apply WASERSTEIN LOSS instead of BINARY CROSS ENTROPY so for BCE object in this case

In [17]:
disc_optimizer = optim.RMSprop(params = discriminator.parameters(), lr = learning_rate)
gen_optimizer = optim.RMSprop(params = generator.parameters(), lr = learning_rate)
real_writer = SummaryWriter(f"logs/real_images")
fake_writer = SummaryWriter(f"logs/fake_images")
fixed_noise = torch.randn(batch_size,z_dim, 1, 1).to(device = device) # to generate fake images later after training the generator 

In [18]:
#The training loop from scratch:

In [ ]:
step = 0
global_tic = time.time()
for epoch in range(EPOCHS):
  tic = time.time()
  print(f"\ntraining begins for epoch: {epoch + 1}\tplease wait........")
  for idx, (data,_) in enumerate(tqdm(loader)):
    data = data.to(device = device)
    noise = torch.randn(batch_size, z_dim, 1,1).to(device = device)
    fake_img = generator(noise)
    #We train the discriminator in the loop (specified frequency)
    for _ in range(disc_iter):
      noise = torch.randn(batch_size, z_dim, 1,1).to(device = device)
      fake_img = generator(noise)
      real_out = discriminator(data).reshape(-1) #flattening everything to 1d
      fake_out = discriminator(fake_img).reshape(-1)
      disc_loss = -(torch.mean(real_out) - torch.mean(fake_out)) #maximize[log(D(x) + D(G(z)))]
      discriminator.zero_grad()#initialize gradients to zero
      disc_loss.backward(retain_graph = True)#discriminator backward pass
      disc_optimizer.step()#gradient descent with RMSprop optimizer for the discriminator
    for m in discriminator.parameters():
      m.data.clamp_(-clip_interval, clip_interval)
    #trainining the generator (its trained fewer number of times compared to discriminator)
    #Here we want to maximize [log(D(G(z)))]
    gen_out = discriminator(fake_img).reshape(-1) #forward pass for the generator
    gen_loss = -torch.mean(gen_out)#Wasserstein loss
    generator.zero_grad()#initialize the gradients to zeros for the generator
    gen_loss.backward()#backward pass for the generator
    gen_optimizer.step()#gradient descent with RMSprop optimizer for the generator
    toc = time.time()

    if idx % 100 ==0:
      print(f"\n>>>>Time elapsed at the end of epoch: {epoch}/{EPOCHS} is {time_fmt(toc - tic)}")
      print(f"\n>>>>Generator loss: {gen_loss:.4f} | Generator PPL: {math.exp(gen_loss):7.4f}")
      print(f">>>>Discriminator loss: {disc_loss:.4f} | Discriminator PPL: {math.exp(disc_loss):7.4f}")

      #saving fake and real images to the specified directories in batches of size 64
      with torch.no_grad():
        fake_image = generator(fixed_noise)
        real_image_grid = torchvision.utils.make_grid(data[:64], normalize = True)
        fake_image_grid = torchvision.utils.make_grid(fake_image[:64], normalize = True)
        real_writer.add_image('Real_images', real_image_grid, global_step = step)
        fake_writer.add_image('Fake_images', fake_image_grid, global_step = step)
      step+=1
global_toc = time.time()
print(f"\n>>>>Total time elapsed for training this model is: {time_fmt(global_toc - global_tic)}")


  0%|          | 0/938 [00:00<?, ?it/s]


training begins for epoch: 1	please wait........

>>>>Time elapsed at the end of epoch: 0/10 is 0 hrs: 00 min: 01.00 sec

>>>>Generator loss: -0.0010 | Generator PPL:  0.9990
>>>>Discriminator loss: -0.4532 | Discriminator PPL:  0.6356


 11%|█         | 100/938 [02:51<23:50,  1.71s/it]


>>>>Time elapsed at the end of epoch: 0/10 is 0 hrs: 02 min: 53.00 sec

>>>>Generator loss: 0.7275 | Generator PPL:  2.0699
>>>>Discriminator loss: -1.6033 | Discriminator PPL:  0.2012


 21%|██▏       | 200/938 [05:42<20:59,  1.71s/it]

...

 53%|█████▎    | 500/938 [14:10<12:27,  1.71s/it]


>>>>Time elapsed at the end of epoch: 8/10 is 0 hrs: 14 min: 12.00 sec

>>>>Generator loss: 0.5105 | Generator PPL:  1.6661
>>>>Discriminator loss: -0.8189 | Discriminator PPL:  0.4409


 64%|██████▍   | 600/938 [17:01<09:36,  1.71s/it]


>>>>Time elapsed at the end of epoch: 8/10 is 0 hrs: 17 min: 03.00 sec

>>>>Generator loss: 0.2920 | Generator PPL:  1.3391
>>>>Discriminator loss: -0.8495 | Discriminator PPL:  0.4276


 71%|███████   | 665/938 [18:52<07:46,  1.71s/it]

In [ ]:
!tensorboard -logdir = logs